In [5]:
import pandas as pd
import numpy as np

In [6]:
string_version = '11.0'
hist_genes = pd.read_csv('histone_genes.csv', sep=',')
lf = pd.read_csv('9606.protein.links.full.v'+string_version+'.txt',sep=' ')

In [8]:
# We make a conversion list from string to HGNC
pa=pd.read_csv('9606.protein.aliases.v'+string_version+'.txt',sep='\t',header=0,names=['id','alias','source'])
pafilt=pa.loc[pa.source.str.contains('BioMart_HUGO( |$)')]
stringid2hgnc=dict(zip(pafilt['id'].tolist(),pafilt['alias'].tolist()))

In [11]:
# Manual adding of gene aliases
stringid2hgnc.update({'9606.ENSP00000339511' : 'HYPM', '9606.ENSP00000301037' : 'RSKR', '9606.ENSP00000439189' : 'POTEE', '9606.ENSP00000345151' : 'RFPL4AL1', '9606.ENSP00000320295' : 'TUBB3', '9606.ENSP00000456953' : 'POTEB2', '9606.ENSP00000454770' : 'ANKRD66', '9606.ENSP00000457423' : 'ZNF625ZNF20', '9606.ENSP00000445077' : 'EIF2S3B', '9606.ENSP00000386655' : 'RPL36AHNRNPH2', '9606.ENSP00000451145' : 'PINX1', '9606.ENSP00000458075' : 'BOLA2SMG1P6', '9606.ENSP00000381081' : 'RNASE4', '9606.ENSP00000233468' : 'SF3B6', '9606.ENSP00000402674': 'BHMG1', '9606.ENSP00000458772' : 'C19orf84', '9606.ENSP00000457511' : 'CCDC179', '9606.ENSP00000299415' : 'CCDC182', '9606.ENSP00000452132' : 'LOC730098', '9606.ENSP00000436891' : 'PRR23D1', '9606.ENSP00000454370' : 'PRR23D2', '9606.ENSP00000454664' : 'SMKR1', '9606.ENSP00000472225' : 'PRR23D2', '9606.ENSP00000425050' : 'CFAP99', '9606.ENSP00000434399' : 'NPIPB8', '9606.ENSP00000431871' : 'NPIPB6', '9606.ENSP00000455944' : 'C3orf84', '9606.ENSP00000409542' : 'CCDC188', '9606.ENSP00000426225' : 'MGAT4D', '9606.ENSP00000466834' : 'RAD51L3RFFL', '9606.ENSP00000355583' : 'MAP3K21', '9606.ENSP00000403980' : 'CLDN34', '9606.ENSP00000403181' : 'C2orf16', '9606.ENSP00000465978' : 'GSG1L2', '9606.ENSP00000313572' : 'NIM1K'})

In [12]:
# Make a reverse dictionary
hgnc2stringid = {v: k for k, v in stringid2hgnc.items()}

In [13]:
#Substitute STRINGIDs for HGNCs in interaction tables
lf['protein1']=[stringid2hgnc.get(x,x) for x in lf['protein1'].values]
lf['protein2']=[stringid2hgnc.get(x,x) for x in lf['protein2'].values]

In [15]:
# Get proteins interacting with histones
hg=hist_genes['HGNC Symbol']
hist_int=lf.loc[lf['protein1'].isin(hg)]

In [16]:
hist_int_1=pd.merge(hist_int, hist_genes[['Histone type', 'HGNC Symbol', 'Canonicity']], how='left', left_on='protein1', right_on='HGNC Symbol')
hist_int_1=hist_int_1.drop(columns=['HGNC Symbol'])

In [17]:
#отфильтровываем интеракторы без названия генов
hist_int_1=hist_int_1.loc[~hist_int_1['protein2'].str.contains('9606.')]

In [18]:
hist_int_1.to_csv('hist_int_1.csv', index=False)

In [19]:
#hist_int_2 - interactors without histones,

hist_int_2=pd.merge(hist_int_1, hist_genes[['HGNC Symbol']], how='left', left_on='protein2', right_on='HGNC Symbol')
hist_int_2=hist_int_2[pd.isnull(hist_int_2['HGNC Symbol'])]
hist_int_2=hist_int_2.drop(columns=['HGNC Symbol'])

In [22]:
# подумать как делать диаграммы венна по трем базам данных, учитывая H1 в гистонах и их интеракторах, пока оставлю без H1
string_interactors_700=hist_int_1['protein2'].loc[(hist_int_1['combined_score']>700)&(hist_int_1['Histone type']!='H1')].drop_duplicates()
string_interactors_700.to_csv('string_interactors_700.csv', index=False, header=True)

string_interactors_700_2=hist_int_2['protein2'].loc[(hist_int_2['combined_score']>700)&(hist_int_2['Histone type']!='H1')].drop_duplicates()
string_interactors_700_2.to_csv('string_interactors_700_2.csv', index=False, header=True)

string_interactors_400=hist_int_1['protein2'].loc[(hist_int_1['combined_score']>400)&(hist_int_1['Histone type']!='H1')].drop_duplicates()
string_interactors_400.to_csv('string_interactors_400.csv', index=False, header=True)

interactions=hist_int_2.loc[hist_int_2['combined_score']>700].drop_duplicates()
interactions=interactions[['protein1', 'protein2', 'combined_score']]
interactions['DB']='String'
interactions.to_csv('string_interaction.csv', index=False, header=True)

In [ ]:
# Here ends processing phase (cell 97 in Annas' script)